<a href="https://colab.research.google.com/github/ssvadla/Demonstration/blob/main/SemiSupervised_Unlabeled_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

train = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [ ]:
Threshold= 0.98

In [ ]:
train['Target'].unique()

array(['Invalid', 'Others', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(3476, 6383)


In [ ]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



X, y = make_classification(random_state=1)

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(x_val)
#print(np.unique(y_pred))
#print(type(y_pred))

Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation Accuracy_score: %f' % Accuracy_score)
test_pred = classifier.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Accuracy test data', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)


Validation Accuracy_score: 0.664823
Accuracy test data 0.6103646833013435
[[ 37   5  13   2   2   5]
 [  0  20   2   1   0   0]
 [ 71  14 432  49  20  32]
 [ 15   6  34  76   6   8]
 [  3   1   2   2  21   4]
 [  0   0   2   2   2  15]]


In [ ]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()


,Unnamed: 0,text,Complete
0,0,No motion for rehearing filed.,1
1,1,— Relator is under accusation of killing her h...,1
2,2,From an order of the District Judge of the 39t...,1
3,3,Relator and her husband had been married for a...,1
4,4,The evidence indicates that their family relat...,1


In [ ]:
del unlabel['Complete']

In [ ]:
del unlabel['Unnamed: 0']

In [ ]:
unlabel.head()

,text
0,No motion for rehearing filed.
1,— Relator is under accusation of killing her h...
2,From an order of the District Judge of the 39t...
3,Relator and her husband had been married for a...
4,The evidence indicates that their family relat...


In [ ]:
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


In [ ]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unlabel.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [ ]:
len(unlabel)

537703

In [ ]:
unlabel_1 = unlabel.loc[:100000]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = unlabel.loc[100001:200000]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = unlabel.loc[200001:300000]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = unlabel.loc[300001:400000]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = unlabel.loc[400001:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 100001
length of unlabel_2 100000
length of unlabel_3 100000
length of unlabel_4 100000
length of unlabel_5 137702


In [ ]:
unlabel_1.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,text
0,responding conclusion regarding evidence annou...
1,conviction transporting intoxicating liquor pu...
2,bill exception case filed late
3,motion new trial overruled october 22 1927 ord...
4,thereafter december 10th extension order made ...


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,text
0,another man carrying clothes
1,officer testified appellant several item cloth...
2,investigation followed report breaking entry a...
3,appellant present
4,warrant arrest issued officer went hotel waite...


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,text
0,statute final judgment appeal court lie senten...
1,sentence must pronounced term court judgment r...
2,court sentence defendant vacation
3,court adjourned authority case ceased notice a...
4,pronouncing sentence authorized vacation autho...


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,text
0,member federal reserve bank
1,opinion state case
2,conviction violating state cigarette law punis...
3,find record bill exception bringing forward co...
4,brief appellant complains lack testimony


In [ ]:
classifier.classes_

array(['Analysis', 'Conclusion', 'Facts', 'Invalid', 'Issue',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['text'])


In [ ]:
x_un1.shape
pred_unlabel_1 = classifier.predict_proba(x_un1)
pred_unlabel_1



array([[0.14416667, 0.26      , 0.28759524, 0.3082381 , 0.        ,
        0.        ],
       [0.04      , 0.02      , 0.36      , 0.53      , 0.05      ,
        0.        ],
       [0.12      , 0.01      , 0.56      , 0.17      , 0.05      ,
        0.09      ],
       ...,
       [0.14      , 0.12      , 0.54      , 0.09      , 0.07      ,
        0.04      ],
       [0.18      , 0.        , 0.52      , 0.2       , 0.        ,
        0.1       ],
       [0.17      , 0.03      , 0.55      , 0.23      , 0.        ,
        0.02      ]])

In [ ]:
print(len(pred_unlabel_1))
print(np.unique(pred_unlabel_1))


100001
[0.         0.00111111 0.00166667 ... 0.986      0.99       1.        ]


In [ ]:
Threshold= 0.98

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[1, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.986, 0.986, 0.99, 0.99, 0.986, 0.99, 0.99, 0.99, 0.99, 0.99, 1.0, 0.986, 1.0, 0.99, 1.0, 0.99, 0.986, 1.0, 1.0, 0.986, 0.986, 0.99, 0.99, 1.0, 0.986, 0.99, 0.986, 1.0, 1.0, 0.99, 0.99, 0.986, 0.986, 1.0, 0.99, 0.99, 0.986, 0.99, 0.99, 0.99, 1.0, 0.986, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 1.0, 1.0, 0.99, 1.0, 0.99, 1.0, 0.99, 0.99, 0.99, 0.986, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 1.0, 0.99, 1.0, 0.99, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 1.0, 0.99, 1.0, 0.99, 1.0, 1.0, 0.99, 0.986, 0.99, 0.986, 0.986, 0.986, 1.0, 0.99, 1.0, 1.0, 0.986, 0.986, 1.0, 1.0, 0.99, 0.99, 1.0, 0.99, 0.99, 0.99, 1.0, 0.

In [ ]:
unlabel_1 = unlabel_1.loc[pos,:]
print(len(unlabel_1))
unlabel_1.head()

114


,text
1480,error mentioned judgment reversed cause remanded
1545,error mentioned judgment reversed cause remanded
2494,witness johnnie bloodworth testified lived wie...
2693,said two men went burglarized house car togeth...
4941,error mentioned judgment reversed cause remanded


In [ ]:
class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

In [ ]:
class_x_un1.shape

(114, 6383)

In [ ]:

class_pred_unlabel_1 = classifier.predict(class_x_un1)
class_pred_unlabel_1


array(['Conclusion', 'Conclusion', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Facts', 'Facts', 'Invalid', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Facts', 'Facts', 'Invalid', 'Facts',
       'Conclusion', 'Facts', 'Invalid', 'Facts', 'Facts', 'Facts',
       'Invalid', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Invalid', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Invalid', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Invalid', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', '

In [ ]:
print(len(class_pred_unlabel_1))

114


In [ ]:
unlabel_1['Target']=class_pred_unlabel_1
unlabel_1.head()

,text,Target
1480,error mentioned judgment reversed cause remanded,Conclusion
1545,error mentioned judgment reversed cause remanded,Conclusion
2494,witness johnnie bloodworth testified lived wie...,Facts
2693,said two men went burglarized house car togeth...,Facts
4941,error mentioned judgment reversed cause remanded,Conclusion


In [ ]:
train = train.rename(columns={'Sentence':'text'})
train.head()

,Unnamed: 0,text,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Invalid
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_1 = [train,unlabel_1]
train_1 = pd.concat(frame_1)
print(len(train))
print(len(unlabel_1))
print(len(train_1))

3476
114
3590


In [ ]:
from sklearn.metrics import classification_report
x_train_1 = tfidf_vect.transform(train_1['text'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1['Target'],test_size=0.25,random_state=2)
classifier_1 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier_1.fit(X_train, Y_train)
y_pred = classifier_1.predict(x_val)
print(np.unique(y_pred))
print(type(y_pred))


Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_1.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred))
print("----------test---------------------")
print(classification_report(test['Target'],test_pred))

['Analysis' 'Conclusion' 'Facts' 'Invalid' 'Issue' 'Rule/Law/Holding']
<class 'numpy.ndarray'>
Validation ------- Accuracy_score: 0.640312
Test ------Accuracy score  0.5988483685220729
[[ 40   9   9   3   3   8]
 [  7  20   2   2   0   0]
 [ 84  14 405  62  31  18]
 [ 15   3  31  76   3   8]
 [  1   0   4   0  19   0]
 [  0   0   0   3   3  15]]
---------validation--------------
                  precision    recall  f1-score   support

        Analysis       0.56      0.27      0.37       147
      Conclusion       0.65      0.43      0.52        46
           Facts       0.66      0.90      0.76       451
         Invalid       0.56      0.52      0.54       146
           Issue       0.79      0.32      0.46        59
Rule/Law/Holding       0.71      0.31      0.43        49

        accuracy                           0.64       898
       macro avg       0.65      0.46      0.51       898
    weighted avg       0.64      0.64      0.61       898

----------test---------------------

Second set of unlabeled data 

In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['text'])


In [ ]:

pred_unlabel_2 = classifier_1.predict_proba(x_un2)
pred_unlabel_2

array([[0.13  , 0.35  , 0.31  , 0.15  , 0.03  , 0.03  ],
       [0.05  , 0.0225, 0.2175, 0.03  , 0.37  , 0.31  ],
       [0.055 , 0.05  , 0.555 , 0.28  , 0.01  , 0.05  ],
       ...,
       [0.05  , 0.    , 0.78  , 0.16  , 0.01  , 0.    ],
       [0.05  , 0.02  , 0.64  , 0.25  , 0.01  , 0.03  ],
       [0.01  , 0.01  , 0.96  , 0.02  , 0.    , 0.    ]])

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[0.         0.0025     0.00333333 ... 0.99       0.99666667 1.        ]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2]
[0.99, 0.99, 1.0, 0.99, 0.99, 1.0, 0.9966666666666667, 0.99, 0.988

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
print(len(unlabel_2))
unlabel_2.head()

311


,text
442,testified knew recognized appellant voice nigh...
609,papa went home got hill dorsey came backi dont...
901,outside saw george foreman come door say hold ...
1073,sheriff moore saw recognized appellant wheel c...
1183,constable testified went appellant home serve ...


In [ ]:
class_x_un2 = tfidf_vect.transform(unlabel_2['text'])

In [ ]:
class_x_un1.shape

(114, 6383)

In [ ]:
class_pred_unlabel_2 = classifier_1.predict(class_x_un2)
class_pred_unlabel_2

array(['Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Conclusion',
       'Conclusion', 'Facts', 'Facts', 'Conclusion', 'Conclusion',
       'Facts', 'Conclusion', 'Conclusion', 'Conclusion', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Conclusion', 'Conclusion', 'Conclusion',
       'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts',
       'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Invalid', 'Facts',
       'Co

In [ ]:
print(len(class_pred_unlabel_2))

311


In [ ]:
unlabel_2['Target']=class_pred_unlabel_2
unlabel_2.head()

,text,Target
442,testified knew recognized appellant voice nigh...,Facts
609,papa went home got hill dorsey came backi dont...,Facts
901,outside saw george foreman come door say hold ...,Facts
1073,sheriff moore saw recognized appellant wheel c...,Facts
1183,constable testified went appellant home serve ...,Facts


In [ ]:
train_1.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_2 = [train_1,unlabel_2]
train_2 = pd.concat(frame_2)
print(len(train_1))
print(len(unlabel_2))
print(len(train_2))

3590
311
3901


In [ ]:
train_2

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
98796,NaN,appellant came back long box brought put car,Facts
98903,NaN,appellant testify offer evidence behalf,Facts
99019,NaN,error pointed judgment reversed cause remanded,Conclusion
99079,NaN,appellant witness testified saw appellant shor...,Facts


In [ ]:
from sklearn.metrics import classification_report
x_train_2 = tfidf_vect.transform(train_2['text'])
x_train_2.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train_2['Target'],test_size=0.25,random_state=2)
classifier_2 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier_2.fit(X_train, Y_train)
y_pred = classifier_2.predict(x_val)
print(np.unique(y_pred))
print(type(y_pred))


Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_2.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred))
print("----------test---------------------")
print(classification_report(test['Target'],test_pred))

['Analysis' 'Conclusion' 'Facts' 'Invalid' 'Issue' 'Rule/Law/Holding']
<class 'numpy.ndarray'>
Validation ------- Accuracy_score: 0.671107
Test ------Accuracy score  0.6065259117082533
[[ 40   9  10   5   8  11]
 [  3  56   1   3   1   0]
 [ 75  14 455  50  24  18]
 [ 18   6  37  72   3  13]
 [  0   1   2   0  16   0]
 [  0   0   0   5   4  16]]
---------validation--------------
                  precision    recall  f1-score   support

        Analysis       0.48      0.29      0.37       136
      Conclusion       0.88      0.65      0.75        86
           Facts       0.72      0.90      0.80       505
         Invalid       0.48      0.53      0.51       135
           Issue       0.84      0.29      0.43        56
Rule/Law/Holding       0.64      0.28      0.39        58

        accuracy                           0.67       976
       macro avg       0.67      0.49      0.54       976
    weighted avg       0.67      0.67      0.65       976

----------test---------------------

Unlabelled 3:

In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['text'])


In [ ]:

pred_unlabel_3 = classifier_2.predict_proba(x_un3)
pred_unlabel_3

array([[0.  , 0.  , 0.77, 0.23, 0.  , 0.  ],
       [0.  , 0.  , 0.48, 0.52, 0.  , 0.  ],
       [0.02, 0.  , 0.97, 0.01, 0.  , 0.  ],
       ...,
       [0.02, 0.01, 0.66, 0.02, 0.13, 0.16],
       [0.12, 0.04, 0.7 , 0.06, 0.05, 0.03],
       [0.  , 0.01, 0.25, 0.67, 0.02, 0.05]])

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[0.         0.0025     0.00333333 ... 0.99       0.99666667 1.        ]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 

In [ ]:
unlabel_3 = unlabel_3.loc[pos,:]
print(len(unlabel_3))
unlabel_3.head()

399


,text
10,crossexamination captain johnson testified wen...
51,error pointed judgment reversed cause remanded
378,track person wearing boot railroad rightofway ...
431,boot appellant fitted track window going samé ...
613,burnett testified saw air conditioning unit si...


In [ ]:
class_x_un3 = tfidf_vect.transform(unlabel_3['text'])

In [ ]:
class_x_un3.shape

(399, 6383)

In [ ]:
class_pred_unlabel_3 = classifier_2.predict(class_x_un3)
class_pred_unlabel_3

array(['Facts', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Invalid', 'Facts',
       'Facts', 'Facts', 'Facts', 'Conclusion', 'Conclusion',
       'Conclusion', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Conclusion', 'Facts',
       'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Conclusion',
       'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Conclusion', 'Conclusion', 'Facts',
       'Facts', 'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Facts',
       'Facts', 'Conclusion', 'Conclusion', 'Facts', 'Conclusion',
       'Conclusion', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Conclusion', 'Facts', 'Conclusion', 'Facts',
       'Conclusion', 'Facts', 'Facts'

In [ ]:
print(len(class_pred_unlabel_3))

399


In [ ]:
unlabel_3['Target']=class_pred_unlabel_3
unlabel_3.head()

,text,Target
10,crossexamination captain johnson testified wen...,Facts
51,error pointed judgment reversed cause remanded,Conclusion
378,track person wearing boot railroad rightofway ...,Facts
431,boot appellant fitted track window going samé ...,Facts
613,burnett testified saw air conditioning unit si...,Facts


In [ ]:
train_2.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_3 = [train_2,unlabel_3]
train_3 = pd.concat(frame_3)
print(len(train_2))
print(len(unlabel_3))
print(len(train_3))

3901
399
4300


In [ ]:
train_3

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
99635,NaN,error pointed judgment reversed cause remanded,Conclusion
99738,NaN,appellant negro company negro also waiting boa...,Facts
99739,NaN,upon approach car mr morris started board car ...,Facts
99845,NaN,front appellant car struck robert lee phillips...,Facts


In [ ]:
from sklearn.metrics import classification_report
x_train_3 = tfidf_vect.transform(train_3['text'])
x_train_3.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train_3['Target'],test_size=0.25,random_state=2)
classifier_3 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier_3.fit(X_train, Y_train)
y_pred = classifier_3.predict(x_val)
print(np.unique(y_pred))
print(type(y_pred))


Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_3.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred))
print("----------test---------------------")
print(classification_report(test['Target'],test_pred))

['Analysis' 'Conclusion' 'Facts' 'Invalid' 'Issue' 'Rule/Law/Holding']
<class 'numpy.ndarray'>
Validation ------- Accuracy_score: 0.709767
Test ------Accuracy score  0.5911708253358925
[[ 38   7   7   4   4   3]
 [  3  77   1   2   1   1]
 [ 78  20 517  54  27  25]
 [ 15   6  26  94   4  12]
 [  1   0   2   1  24   0]
 [  0   0   1   4   3  13]]
---------validation--------------
                  precision    recall  f1-score   support

        Analysis       0.60      0.28      0.38       135
      Conclusion       0.91      0.70      0.79       110
           Facts       0.72      0.93      0.81       554
         Invalid       0.60      0.59      0.59       159
           Issue       0.86      0.38      0.53        63
Rule/Law/Holding       0.62      0.24      0.35        54

        accuracy                           0.71      1075
       macro avg       0.72      0.52      0.58      1075
    weighted avg       0.71      0.71      0.68      1075

----------test---------------------

Unlabeled 4

In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['text'])


In [ ]:

pred_unlabel_4 = classifier_3.predict_proba(x_un4)
pred_unlabel_4

array([[0.26      , 0.08      , 0.25      , 0.35      , 0.        ,
        0.06      ],
       [0.11      , 0.09      , 0.45      , 0.25      , 0.03      ,
        0.07      ],
       [0.05      , 0.07      , 0.73      , 0.14      , 0.        ,
        0.01      ],
       ...,
       [0.25      , 0.03      , 0.59      , 0.12      , 0.01      ,
        0.        ],
       [0.2875    , 0.03      , 0.4525    , 0.19      , 0.03      ,
        0.01      ],
       [0.27028571, 0.71971429, 0.        , 0.        , 0.01      ,
        0.        ]])

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[0.         0.0025     0.00333333 ... 0.99       0.99666667 1.        ]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 

In [ ]:
unlabel_4 = unlabel_4.loc[pos,:]
print(len(unlabel_4))
unlabel_4.head()

699


,text
56,w k mcneir witness state testified state ridin...
76,error pointed judgment reversed cause remanded
169,saw appellant behind steering wheel car
189,appellant low blood pressure loss blood bleedi...
199,one car light went face went left trying turn ...


In [ ]:
class_x_un4 = tfidf_vect.transform(unlabel_4['text'])

In [ ]:
class_x_un4.shape

(699, 6383)

In [ ]:
class_pred_unlabel_4 = classifier_3.predict(class_x_un4)
class_pred_unlabel_4

array(['Facts', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Invalid', 'Conclusion',
       'Invalid', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Conclusion', 'Facts', 'Facts', 'Facts', 'Invalid', 'Facts',
       'Facts', 'Conclusion', 'Conclusion', 'Conclusion', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion',
       'Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Conclusion', 'Facts',
       'Facts', 'Facts', 'Conclusion', 'Facts', 'Facts', 'Facts', 'Facts',
       'Facts', 'Invalid', 'Facts', 'Facts', 'Facts', 'Facts', 'F

In [ ]:
print(len(class_pred_unlabel_4))

699


In [ ]:
unlabel_4['Target']=class_pred_unlabel_4
unlabel_4.head()

,text,Target
56,w k mcneir witness state testified state ridin...,Facts
76,error pointed judgment reversed cause remanded,Conclusion
169,saw appellant behind steering wheel car,Facts
189,appellant low blood pressure loss blood bleedi...,Facts
199,one car light went face went left trying turn ...,Facts


In [ ]:
train_3.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_4 = [train_3,unlabel_4]
train_4 = pd.concat(frame_4)
print(len(train_3))
print(len(unlabel_4))
print(len(train_4))
train_4

4300
699
4999


,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
99761,NaN,massey drunk went sleep bed oneroom house occu...,Facts
99765,NaN,third shot fired appellant left home truck lat...,Facts
99837,NaN,soon crowd saw appellant door broke get porter...,Facts
99885,NaN,affidavit one two negro vestibule rear baggage...,Facts


In [ ]:
from sklearn.metrics import classification_report
x_train_4 = tfidf_vect.transform(train_4['text'])
x_train_4.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train_4['Target'],test_size=0.25,random_state=2)
classifier_4 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier_4.fit(X_train, Y_train)
y_pred = classifier_4.predict(x_val)
print(np.unique(y_pred))
print(type(y_pred))


Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_4.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred))
print("----------test---------------------")
print(classification_report(test['Target'],test_pred))

['Analysis' 'Conclusion' 'Facts' 'Invalid' 'Issue' 'Rule/Law/Holding']
<class 'numpy.ndarray'>
Validation ------- Accuracy_score: 0.756800
Test ------Accuracy score  0.6026871401151631
[[ 33   9   7   3   5   4]
 [  1 108   4   1   1   0]
 [ 90  14 673  44  21  23]
 [ 11   6  38  96   5   7]
 [  1   0   1   0  18   1]
 [  2   0   1   2   2  18]]
---------validation--------------
                  precision    recall  f1-score   support

        Analysis       0.54      0.24      0.33       138
      Conclusion       0.94      0.79      0.86       137
           Facts       0.78      0.93      0.85       724
         Invalid       0.59      0.66      0.62       146
           Issue       0.86      0.35      0.49        52
Rule/Law/Holding       0.72      0.34      0.46        53

        accuracy                           0.76      1250
       macro avg       0.74      0.55      0.60      1250
    weighted avg       0.75      0.76      0.73      1250

----------test---------------------

Unlabeld 5

In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['text'])


In [ ]:
x_un5.shape

(137702, 6383)

In [ ]:
pred_unlabel_5 = classifier_4.predict_proba(x_un5)
pred_unlabel_5

array([[0.01      , 0.08      , 0.03      , 0.78      , 0.04      ,
        0.06      ],
       [0.03      , 0.14      , 0.18      , 0.61      , 0.        ,
        0.04      ],
       [0.05      , 0.04      , 0.25      , 0.05      , 0.27333333,
        0.33666667],
       ...,
       [0.37      , 0.01      , 0.16      , 0.46      , 0.        ,
        0.        ],
       [0.08      , 0.06      , 0.65      , 0.08      , 0.08      ,
        0.05      ],
       [0.29333333, 0.36666667, 0.12      , 0.21      , 0.        ,
        0.01      ]])

In [ ]:
print(len(pred_unlabel_5))
print(np.unique(pred_unlabel_5)) 

137702
[0.         0.00142857 0.002      ... 0.98666667 0.99       1.        ]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 3, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]
print(len(unlabel_5))
unlabel_5.head()

1257


,text
182,appellant testified shortly thereafter decease...
218,effort get possession wrench appellant would p...
225,error pointed judgment reversed cause remanded
359,witness r maynard testified one occasion saw e...
445,arrived whisky sometime 730 1000 oclock night ...


In [ ]:
class_x_un5 = tfidf_vect.transform(unlabel_5['text'])

In [ ]:
class_x_un5.shape

(1257, 6383)

In [ ]:
class_pred_unlabel_5 = classifier_4.predict(class_x_un5)
class_pred_unlabel_5

array(['Facts', 'Facts', 'Conclusion', ..., 'Facts', 'Facts',
       'Conclusion'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_5))

1257


In [ ]:
unlabel_5['Target']=class_pred_unlabel_5
unlabel_5.head()

,text,Target
182,appellant testified shortly thereafter decease...,Facts
218,effort get possession wrench appellant would p...,Facts
225,error pointed judgment reversed cause remanded,Conclusion
359,witness r maynard testified one occasion saw e...,Facts
445,arrived whisky sometime 730 1000 oclock night ...,Facts


In [ ]:
train_4.head()


,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
print(len(train_4))

4999


In [ ]:
frame_5 = [train_4,unlabel_5]
train_5 = pd.concat(frame_5)
print(len(train_4))
print(len(unlabel_5))
print(len(train_5))

4999
1257
6256


In [ ]:
from sklearn.metrics import classification_report
x_train_5 = tfidf_vect.transform(train_5['text'])
x_train_5.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train_5['Target'],test_size=0.25,random_state=2)
classifier_5 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier_5.fit(X_train, Y_train)
y_pred = classifier_5.predict(x_val)
print(np.unique(y_pred))
print(type(y_pred))


Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation ------- Accuracy_score: %f' % Accuracy_score)

test_pred = classifier_5.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Test ------Accuracy score ', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)
print("---------validation--------------")
print(classification_report(y_val,y_pred))
print("----------test---------------------")
print(classification_report(test['Target'],test_pred))

['Analysis' 'Conclusion' 'Facts' 'Invalid' 'Issue' 'Rule/Law/Holding']
<class 'numpy.ndarray'>
Validation ------- Accuracy_score: 0.794118
Test ------Accuracy score  0.6180422264875239
[[ 36   9   7   3   4   3]
 [  2 149   2   0   0   1]
 [ 86  14 916  44  32  27]
 [ 18   0  40 104   7  10]
 [  2   0   2   1  16   2]
 [  1   0   1   0   4  21]]
---------validation--------------
                  precision    recall  f1-score   support

        Analysis       0.58      0.25      0.35       145
      Conclusion       0.97      0.87      0.91       172
           Facts       0.82      0.95      0.88       968
         Invalid       0.58      0.68      0.63       152
           Issue       0.70      0.25      0.37        63
Rule/Law/Holding       0.78      0.33      0.46        64

        accuracy                           0.79      1564
       macro avg       0.74      0.55      0.60      1564
    weighted avg       0.78      0.79      0.77      1564

----------test---------------------